---
title: $k$-Means Clustering
jupyter: python3
---

In [ ]:
#| echo: false
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Victorian England


## 1854 Cholera Outbreak

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tools4ds/DS701-Course-Notes/blob/main/ds701_book/jupyter_notebooks/06-Clustering-I-kmeans.ipynb)

:::: {.columns }

::: {.column width="40%"}
![London cholera outbreak](./figs/Punch-A_Court_for_King_Cholera.png){width=400}
^[Public Domain, https://commons.wikimedia.org/w/index.php?curid=680455]
:::

::: {.column width="60%" .incremental}
- There was a horrific cholera outbreak in **1854** Soho, London.
- Common wisdom at the time was that disease spread by breathing "foul air" (miasma).  
- The London sewer system had not yet reached Soho.  
- Most homes had cesspits under the floor, which often overflowed.
- "Night Soil Men" would regularly collect and sell to farmers or dump in the Thames.
:::

::::

## John Snow

:::: {.columns }

::: {.column width="60%" .incremental}
- John Snow, a local physician, extensively studied the patterns of illness across Soho due to cholera.
- In the course of his studies, his attention was drawn to one neighborhood around Broad Street.
- In 10 days, 500 people in the area died. 
:::

::: {.column width="40%"}
![John Snow](./figs/L06-John-Snow-Portrait.png)
:::

::::

## John's Snow Map

:::: {.columns }

::: {.column width="60%"}
![London cholera outbreak](figs/L6-Snow-cholera-map-1.png){width=500px}
:::

::: {.column width="40%" .incremental}
- In uncovering the source of this outbreak, Snow prepared this map.
- From this map he could clearly see the deaths were clustered around an area.
- The neighborhood was all served by a water pump on Broad St.
- The pump handle was removed and illnesses decreased dramatically.

::: {.fragment}
![Broad St. water pump](./figs/L6-John_Snow_memorial_and_pub.png){width=180}
:::

:::

::::

---

:::: {.columns}
::: {.column width="50%"}
![J. Snow Book](figs/L06-snow-book-cover.png){height=500}
:::
::: {.column width="50%" .incremental}
- He later published his results^[By John Snow - Published by C.F. Cheffins, Lith, Southhampton Buildings, London, England, 1854 in Snow, John. On the Mode of Communication of Cholera, 2nd Ed, John Churchill, New Burlington Street, London, England, 1855.]
- Results from the cluster map.
- Results from a double blind study of two neighborhoods drawing water upriver and downriver of the polluted portion of the Thames.
- Other anecdotes of visitors to Soho, etc.
:::
::::

::: {.content-hidden when-profile="slides"}
## References and Further Reading 

Images and information taken from [wikipedia](https://en.wikipedia.org/wiki/1854_Broad_Street_cholera_outbreak), 
[National Library of Medicine](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7150208/) and the 
[Internet Archive](https://archive.org/details/b28985266/page/n3/mode/2up) and
[MSU's John Snow Archive](https://johnsnow.matrix.msu.edu/).

John Snow's original data is recreated [here](https://blog.rtwilson.com/john-snows-cholera-data-in-more-formats/).
:::

# Clustering

## Clustering

::: {.fragment}
Clustering is a very important way of discovering __structure__ in data.
:::

::: {.fragment}
It is so important because it is __common__ for data to show clusters.
:::

::: {.incremental}
* Locations where millionaires live
* The number of hours people work each week
* Demographics ("soccer parents", "bored retirees", "unemployed millenials", etc)
:::

::: {.content-hidden when-profile="web"}
## Clustering Continued
:::

::: {.incremental}
* We can often simplify or compress our data if we recognize the existence of clusters.
* Further, we can often interpret clusters by assigning them labels.
* However, note that these categories or "labels" are assigned __after__ the fact.
* And, we may __not__ be able to interpret clusters or assign them labels in some cases.
* That is, clustering represents the first example we will see of __unsupervised__ learning.
:::

## Supervised vs Unsupervised

__Supervised__ methods:  Data items have labels, and we want to learn a function that correctly assigns labels to new data items.

__Unsupervised__ methods:  Data items do not have labels, and we want to learn a function that extracts  important patterns from the data.

## Applications of Clustering

* Image Processing
    * Cluster images based on their visual content.
    * Compress images based on color clusters.
* Web Mining
    * Cluster groups of users based on webpage access patterns.
    * Cluster web pages based on their content.
* Bioinformatics
    * Cluster similar proteins together (by structure or function).
    * Cluster cell types (by gene activity).
* And many more ...

## The Clustering Problem

When we apply clustering, what problem are we trying to solve?

::: {.fragment}
We will answer this question informally at first, then apply formal criteria.
:::

::: {.fragment .fade-in}
Informally, a __clustering__ is:
    
> a grouping of data objects, such that the objects within a group are similar (or near) to one another and dissimilar (or far) from the objects in other groups.

(keep in mind that if we use a distance function as a dissimilarity measure, then "far" implies "different")
:::

::: {.content-hidden when-profile="web"}
## The Clustering Problem, continued
:::


:::: {.columns}
::: {.column width="50%"}
So we want our clustering algorithm to:

* <font color = "blue">minimize</font> intra-cluster distances.
* <font color = "red">maximize</font> inter-cluster distances.
:::
::: {.column width="50%"}
![](figs/L06-clustering-1.png){width=500 fig-align="center"}
:::
::::


## The Clustering Problem, continued

:::: {.columns}
::: {.column width="50%"}
Here are the basic questions we need to ask about clustering:

::: {.incremental}
* What is the right kind of <font color = "blue">"similarity"</font> to use?
* What is a <font color="blue">*good* partition</font> of objects?
    * i.e., how is the quality of a solution measured?
* <font color = "blue">How to find</font> a good partition?
    * Are there efficient algorithms?  
    * Are there algorithms that are guaranteed to find good clusters?
:::
:::
::: {.column width="50%"}
![](figs/L06-clustering-1.png){width=500 fig-align="center"}
:::
::::


::: {.content-hidden when-profile="web"}
## The Clustering Problem, continued
:::

Now note that even with our more-formal discussion, the criteria for deciding on
a "best" clustering can still be ambiguous.
    
![](figs/L06-clustering-2.png){width=600 fig-align="center"}

::: {.fragment}
Later, we will discuss hierarchical clustering which let's us consider clustering
at multiple scales.
:::

## Partitional Clustering

:::: {.columns}
::: {.column width="50%"}
For now, we'll focus on __partitional__ clustering. 

Points are divided into a set of __non-overlapping__ groups:

* Each object belongs to __one__, and only one, cluster. (mutually exclusive)
* The set of clusters covers all the objects. (exhaustive)

We are going to assume for now that the number of clusters, $k$, is given in advance.
:::
::: {.column width="50%"}
![](figs/L06-partitional-clustering.png){width=600 fig-align="center"}
:::
::::

# The $k$-means Algorithm

## Assumptions

Now, we are ready to state our first formalization of the clustering problem.

We will assume that 

* data items are represented by points in $d$-dimensional space, $\mathbb{R}^d$, i.e.,  has $d$ features,
* the number of points, $N$, is given, and
* the number of clusters $K$ is given.

## Minimizing a Cost Function

Find $K$ disjoint clusters $C_k$, each described by points $c_1, \dots, c_K$
(called <font color="blue"> _centers_, _centroids_,</font> or <font color = "blue"> _means_)</font>
that minimizes

$$
\underset{C_1, \dots, C_K}{\arg\min} \hspace{1em} \sum_{k=1}^K \hspace{1em} \sum_{x\in C_k} \Vert x-c_k\Vert^2. 
$$

* The _Within-Cluster Sum of Squares_ (WCSS) or the _Inertia_ of the clustering.

::: aside
See [Norms](./05-Distances-Timeseries.html#norms) in
[Distances and Timeseries](./05-Distances-Timeseries.html) for a refresher.
:::

---

We now have a <font color="blue">formal</font> definition of a clustering.

This is not the only definition possible, but it is an intuitive and simple one.

How hard is it to solve this problem?

* $k=1$ and $k=n$ are easy special cases. Why?
* But, this problem is __NP-hard__ if the dimension of the data is at least 2.
    * We don't expect that there is any exact, efficient algorithm in general.

__Nonetheless,__ there is a simple algorithm that works quite well in practice!


## Aside: $k$-means as NP-hard

For context, **NP-hard** is a term used in the study of
[computational complexity](https://en.wikipedia.org/wiki/Computational_complexity_theory).

Problems in [**P**](https://en.wikipedia.org/wiki/P_(complexity)) are those that
can be solved in polynomial time,  e.g. for $n$ items, the time to solve the problem is $O(n^2)$ or $O(n^3)$.

Problems in [**NP**](https://en.wikipedia.org/wiki/NP_(complexity)) are those that
can be _verified_ in polynomial time but not necessarily _solved_ in polynomial time.

[**NP-hard**](https://en.wikipedia.org/wiki/NP-hardness) problems are those that are at least as hard as the hardest problems in **NP**, not necessarily verifiable in polynomial time.

[**NP-complete**](https://en.wikipedia.org/wiki/NP-complete) problems are those that are both NP-hard and in NP, e.g. hardest problems in NP and verifiable in polynomial time.

You can prove that the $k$-means problem is NP-hard by finding a reduction from a known NP-hard problem such as the [**Partition Problem**](https://en.wikipedia.org/wiki/Partition_problem).


## The $k$-means Algorithm

:::: {.columns }

::: {.column width="40%"}
![](figs/L06-top-ten-algorithms-cover.png){width=300 fig-align="center"}
:::

::: {.column width="60%"}
* There is a "classic" algorithm for this problem.
* It was voted among the __top-10 algorithms__ in data mining!^[As determined at the 2006 IEEE International Conference on Data Mining]
* It is such a good idea that it has been independently discovered multiple times.
* It was first discovered by Lloyd in 1957, so it is often called Lloyd's algorithm.
* It is called the "$k$-means algorithm."
* Not to be confused with the $k$-means problem of which this is a heuristic solution.
:::

::::

::: aside
The other from the top 10 were SVM, Apriori, EM, PageRank, AdaBoost, kNN, Naive Bayes, and CART.
:::

::: {.content-hidden when-profile="web"}
## The $k$-means algorithm
:::

1. Pick $K$ cluster centers $\{c_1, \dots, c_K\}$.  
    - These can be randomly chosen data points, or by some other method such as
     [$k$-means++](https://en.wikipedia.org/wiki/K-means%2B%2B)
2. For each $j$, define the cluster $C_j$ as the set of points in $X$ that are <font color="blue">closest to center</font> $c_j$.  
    - Nearer to $c_j$ than to any other center.
3. For each $j$, update $c_j$ to be <font color="blue">the center of mass of cluster</font> $C_j$.  
    - In other words, $c_j$ is the mean of the vectors in $C_j$.
4. Repeat (i.e., go to Step 2) until convergence.
    - Either the cluster centers change below a threshold, or inertia changes below a threshold or a maximum number of iterations is reached.

---

Let's see this in practice with well separated clusters and also look at the Within-Cluster Sum of Square (WCSS).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Set random seed for reproducibility
np.random.seed(42)

# Generate sample data
n_samples = 300
X, _ = make_blobs(n_samples=n_samples, centers=3, cluster_std=0.60, random_state=42)

# Initialize centroids randomly
k = 3
centroids = X[np.random.choice(n_samples, k, replace=False)]

# Function to assign points to clusters
def assign_clusters(X, centroids):
    distances = np.sqrt(((X - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

# Function to update centroids
def update_centroids(X, labels, k):
    return np.array([X[labels == i].mean(axis=0) for i in range(k)])

# Function to calculate within-cluster sum of squares
def calculate_wcss(X, labels, centroids):
    return sum(np.sum((X[labels == i] - centroids[i])**2) for i in range(k))

# Set up the clustering progress plot
fig1, axs = plt.subplots(2, 3, figsize=(10, 6))
axs = axs.ravel()

# Colors for each cluster
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# List to store WCSS values
wcss_values = []

# Run k-means iterations and plot
for i in range(6):
    # Assign clusters
    labels = assign_clusters(X, centroids)
    
    # Calculate WCSS
    wcss = calculate_wcss(X, labels, centroids)
    wcss_values.append(wcss)
    
    # Plot the current clustering state
    axs[i].scatter(X[:, 0], X[:, 1], c=[colors[l] for l in labels], alpha=0.6)
    axs[i].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=200, linewidths=3)
    axs[i].set_title(f'Iteration {i if i < 5 else "Final"}, WCSS: {wcss:.2f}')
    axs[i].set_xlim(X[:, 0].min() - 1, X[:, 0].max() + 1)
    axs[i].set_ylim(X[:, 1].min() - 1, X[:, 1].max() + 1)
    
    # Update centroids (except for the last iteration)
    if i < 5:
        centroids = update_centroids(X, labels, k)

plt.tight_layout()

# Create a separate plot for WCSS progress
fig2, ax = plt.subplots(figsize=(10, 6))
ax.plot(range(6), wcss_values, marker='o')
ax.set_title('WCSS Progress')
ax.set_xlabel('Iteration')
ax.set_ylabel('Within-Cluster Sum of Squares')
ax.grid(True)

plt.tight_layout()
plt.show()

---

Let's see this in practice with overlapping clusters and also look at the WCSS.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Set random seed for reproducibility
np.random.seed(42)

# Generate sample data
n_samples = 300
X, _ = make_blobs(n_samples=n_samples, centers=3, cluster_std=3.00, random_state=2)

# Initialize centroids randomly
k = 3
centroids = X[np.random.choice(n_samples, k, replace=False)]

# Function to assign points to clusters
def assign_clusters(X, centroids):
    distances = np.sqrt(((X - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

# Function to update centroids
def update_centroids(X, labels, k):
    return np.array([X[labels == i].mean(axis=0) for i in range(k)])

# Function to calculate within-cluster sum of squares
def calculate_wcss(X, labels, centroids):
    return sum(np.sum((X[labels == i] - centroids[i])**2) for i in range(k))

# Set up the clustering progress plot
fig1, axs = plt.subplots(2, 3, figsize=(10, 6))
axs = axs.ravel()

# Colors for each cluster
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# List to store WCSS values
wcss_values = []

# Run k-means iterations and plot
for i in range(6):
    # Assign clusters
    labels = assign_clusters(X, centroids)
    
    # Calculate WCSS
    wcss = calculate_wcss(X, labels, centroids)
    wcss_values.append(wcss)
    
    # Plot the current clustering state
    axs[i].scatter(X[:, 0], X[:, 1], c=[colors[l] for l in labels], alpha=0.6)
    axs[i].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=200, linewidths=3)
    axs[i].set_title(f'Iteration {i if i < 5 else "Final"}, WCSS: {wcss:.2f}')
    axs[i].set_xlim(X[:, 0].min() - 1, X[:, 0].max() + 1)
    axs[i].set_ylim(X[:, 1].min() - 1, X[:, 1].max() + 1)
    
    # Update centroids (except for the last iteration)
    if i < 5:
        centroids = update_centroids(X, labels, k)

plt.tight_layout()

# Create a separate plot for WCSS progress
fig2, ax = plt.subplots(figsize=(10, 6))
ax.plot(range(6), wcss_values, marker='o')
ax.set_title('WCSS Progress')
ax.set_xlabel('Iteration')
ax.set_ylabel('Within-Cluster Sum of Squares')
ax.grid(True)

plt.tight_layout()
plt.show()

---

## Limitations of $k$-means

As you can see, $k$-means can work very well.

However, we don't have any guarantees on the performance of $k$-means.

In particular, there are various settings in which $k$-means can fail to do a good job.


## Limitation: non-spherical clusters

Because each point is assigned to its closest center, the points in a cluster are implicitly assumed to be arranged in a sphere around the center.

In [ ]:
#| fig-cap: "K-means clustering on a dataset with anisotropic clusters."
#| fig-align: "center"

# Author: Phil Roth <mr.phil.roth@gmail.com>
#         Arturo Amor <david-arturo.amor-quiroz@inria.fr>
# License: BSD 3 clause
#
# From https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html

import numpy as np
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Generate two half-moon clusters
X_moons, y_moons_true = make_moons(n_samples=200, noise=0.1, random_state=42)

# Perform k-means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
y_moons_pred = kmeans.fit_predict(X_moons)

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot ground truth
ax1.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons_true, cmap='viridis')
ax1.set_title('Ground Truth')
ax1.set_xlabel('Feature 1')
ax1.set_ylabel('Feature 2')

# Plot k-means clustering result
ax2.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons_pred, cmap='viridis')
ax2.set_title('K-means Clustering')
ax2.set_xlabel('Feature 1')
ax2.set_ylabel('Feature 2')

# Add cluster centers to the k-means plot
ax2.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

plt.tight_layout()
plt.show()

## Limitation: anisotropic clusters

In [ ]:
#| fig-cap: "K-means clustering on a dataset with anisotropic gaussian clusters."

import numpy as np
from sklearn.datasets import make_blobs

n_samples = 1500
random_state = 170
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]

X, y = make_blobs(n_samples=n_samples, random_state=random_state)
X_aniso = np.dot(X, transformation)  # Anisotropic blobs
X_varied, y_varied = make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
    )  # Unequal variance
X_filtered = np.vstack(
    (X[y == 0][:500], X[y == 1][:100], X[y == 2][:10])
    )  # Unevenly sized blobs
y_filtered = [0] * 500 + [1] * 100 + [2] * 10

# run previous two cells first

common_params = {
    "n_init": "auto",
    "random_state": random_state,
}

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X_aniso[:, 0], X_aniso[:, 1], c=y)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=3, **common_params)
y_pred = kmeans.fit_predict(X_aniso)
axs[1].scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

# plt.suptitle("Unexpected KMeans clusters").set_y(0.95)
plt.show()

## Limitation: unequal-sized clusters

For the same reason, $k$-means tends to try to balance the sizes of the clusters.

In [ ]:
#| fig-cap: "K-means clustering on a dataset with unequal-sized clusters."

# run previous three cells first

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_filtered)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=3, **common_params)
y_pred = kmeans.fit_predict(X_filtered)
axs[1].scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

# plt.suptitle("Unexpected KMeans clusters").set_y(0.95)
plt.show()

## Limitation: unequal variance clusters

In [ ]:
#| fig-cap: "K-means clustering on a dataset with unequal variance clusters."

# run previous four cells first

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X_varied[:, 0], X_varied[:, 1], c=y_varied)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=3, **common_params)
y_pred = kmeans.fit_predict(X_varied)
axs[1].scatter(X_varied[:, 0], X_varied[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

# plt.suptitle("Unexpected KMeans clusters").set_y(0.95)
plt.show()

## Limitation: sensitive to starting cluster centers

If the initial guess (Step 1) is a bad one, $k$-means may get "stuck" in a bad solution.
    
![](figs/L06-kmeans-bad-initialization.png){width=600 fig-align="center"}


## Limitation: sensitive to number of clusters

In [ ]:
#| fig-cap: "K-means clustering on a dataset with incorrect number of clusters parameter."

# run previous five cells first

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X[:, 0], X[:, 1], c=y)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=2, **common_params)
y_pred = kmeans.fit_predict(X)
axs[1].scatter(X[:, 0], X[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

plt.show()

## Choosing a Good Initialization

* How can we avoid the kind of bad initialization we just saw?
* A good strategy is to pick points that are distant to each other.
* This strategy is called ["$k$-means++"](https://en.wikipedia.org/wiki/K-means%2B%2B).  
* It works very well in practice, and the `scikit-learn` implementation uses it by default.
* We will explore it in more detail in the next lecture.

## Choosing the right $k$

Given some $K$, the $k$-means algorithm "learns" the cluster centers.

::: {.fragment}
How do we choose the right number of clusters, $K$?
:::

::: {.fragment}
As an aside:

* This parameter ($K$) is the first example we have seen of a __hyperparameter__.
* A hyperparameter is a parameter that must be set before the model parameters can be learned.
:::

::: {.fragment}
Our basic strategy will be to:

* Iterate through different $K$ and use some criterion to decide which $K$ is most appropriate.
* We will discuss this more in the next lecture.
:::

## Feature Scales

Finally, given the tendency of $k$-means to look for spherical clusters, we should consider the scales of the various features.

In fact, in general when constructing or selecting a distance metric, one needs to think carefully about the scale of the features being used.

## Unscaled Features

For example, consider the case where we are clustering people based on their age, income, and gender.

We might use age in years, income in dollars, and assign gender to the values $\{0, 1\}$.

Thus, the following records:

* Joe Smith, age 27, income USD 75,000, male
* Eve Jones, age 45, income USD 42,000, female

Would be encoded in feature space as:

$$
\begin{bmatrix}27\\75000\\0\end{bmatrix},\begin{bmatrix}45\\42000\\1\end{bmatrix} 
$$

## Unscaled Features, Continued

What would happen if we used Euclidean distance as our dissimilarity metric in this feature space?

(This is what $k$-means uses.)

:::: {.fragment}
Clearly, the influence of income would dominate the other two features.  For
example, a difference of gender is about as significant as a difference of one
dollar of yearly income.

We are unlikely to expose gender-based differences if we cluster using this representation.

The most common way to handle this is __feature scaling.__
::::

## Feature Scaling

The basic idea is to rescale each feature separately, so that its range of values is about the same as all other features.

For example, one may choose to:
    
* shift each feature independently by subtracting the mean over all observed values.
    * This means that each feature is now centered on zero.
* Then rescale each feature so that the standard deviation overall observed values is 1.
    * This means that the feature will have about the same range of values as all the others.

# $k$-means Clustering Examples

# Example 1: Customer Segmentation

## Customer Segmentation

Another powerful application of $k$-means clustering is **customer segmentation** in e-commerce.

::: {.incremental}
* Businesses want to understand their customers to provide better service and targeted marketing
* Group customers with similar behaviors and characteristics $\rightarrow$ **personas**
* This enables personalized marketing campaigns, product recommendations, and customer service strategies
:::

## E-commerce Customer Data

Let's synthesize e-commerce data to segment customers based on their purchasing behavior.

High-value, frequent buyers; Moderate buyers; Bargain hunters; Occasional buyers.

| Segment | Percentage | Annual Spending | Average Order Value | Total Orders |
|---------|------------|----------------|---------------------|--------------|
| High-value | 20% | $120 \pm 20$ | $15 \pm 3$ | $450 \pm 80$ |
| Moderate | 40% | $60 \pm 15$ | $8 \pm 2$ | $250 \pm 50$ |
| Bargain | 25% | $25 \pm 8$ | $12 \pm 4$ | $150 \pm 30$ |
| Occasional | 15% | $15 \pm 5$ | $3 \pm 1$ | $300 \pm 60$ |

## E-commerce Customer Data

In [ ]:
#| fig-cap: "Customer segmentation analysis using e-commerce data"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Create realistic e-commerce customer data
np.random.seed(42)
n_customers = 1000

# Generate customer segments with different characteristics
# Segment 1: High-value, frequent buyers (20% of customers)
high_value = np.random.normal([120, 15, 450], [20, 3, 80], (200, 3))

# Segment 2: Moderate buyers (40% of customers)
moderate = np.random.normal([60, 8, 250], [15, 2, 50], (400, 3))

# Segment 3: Bargain hunters (25% of customers)
bargain = np.random.normal([25, 12, 150], [8, 4, 30], (250, 3))

# Segment 4: Occasional buyers (15% of customers)
occasional = np.random.normal([15, 3, 300], [5, 1, 60], (150, 3))

# Combine all segments
customer_data = np.vstack([high_value, moderate, bargain, occasional])

# Add some noise and shuffle
customer_data += np.random.normal(0, 5, customer_data.shape)
np.random.shuffle(customer_data)

# Create DataFrame
df = pd.DataFrame(customer_data, columns=['Annual_Spending', 'Avg_Order_Value', 'Total_Orders'])
df['Customer_ID'] = range(1, len(df) + 1)

print("E-commerce Customer Data Head:")
print(df.head().round(2))


print("\n\nE-commerce Customer Data Summary    :")
print(df.describe().round(2))

## Understanding Customer Features

In [ ]:
#| fig-cap: "Distribution of customer features"

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Annual Spending
axes[0].hist(df['Annual_Spending'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Distribution of Annual Spending')
axes[0].set_xlabel('Annual Spending ($)')
axes[0].set_ylabel('Number of Customers')

# Average Order Value
axes[1].hist(df['Avg_Order_Value'], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1].set_title('Distribution of Average Order Value')
axes[1].set_xlabel('Average Order Value ($)')
axes[1].set_ylabel('Number of Customers')

# Total Orders
axes[2].hist(df['Total_Orders'], bins=30, alpha=0.7, color='salmon', edgecolor='black')
axes[2].set_title('Distribution of Total Orders')
axes[2].set_xlabel('Total Orders')
axes[2].set_ylabel('Number of Customers')

plt.tight_layout()
plt.show()

> Can you see clusters?

## Feature Scaling is Critical

Before clustering, we need to scale our features since they have very different ranges:

In [ ]:
#| fig-cap: "Feature scaling for customer segmentation"

# Prepare features for clustering
features = ['Annual_Spending', 'Avg_Order_Value', 'Total_Orders']
X = df[features].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Original feature ranges:")
print(f"Annual Spending: ${X[:, 0].min():.0f} - ${X[:, 0].max():.0f}")
print(f"Avg Order Value: ${X[:, 1].min():.0f} - ${X[:, 1].max():.0f}")
print(f"Total Orders: {X[:, 2].min():.0f} - {X[:, 2].max():.0f}")

print("\nScaled feature ranges:")
print(f"Annual Spending: {X_scaled[:, 0].min():.2f} - {X_scaled[:, 0].max():.2f}")
print(f"Avg Order Value: {X_scaled[:, 1].min():.2f} - {X_scaled[:, 1].max():.2f}")
print(f"Total Orders: {X_scaled[:, 2].min():.2f} - {X_scaled[:, 2].max():.2f}")

## Searching Cluster Numbers

What cluster number should we pick?

In [ ]:
#| fig-cap: "Elbow method for determining optimal number of clusters"

# Calculate WCSS for different numbers of clusters
wcss = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, wcss, 'bo-', linewidth=2, markersize=8)
plt.title('WCSS vs Number of Clusters')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Customer Segmentation Results

Visualize in 2D using PCA which we'll cover later.

In [ ]:
#| fig-cap: "K-means customer segmentation results"

# Perform k-means clustering with k=4
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Add cluster labels to the dataframe
df['Cluster'] = cluster_labels

# Visualize clusters in 2D using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(12, 5))

# Plot 1: PCA visualization
plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            c='red', marker='x', s=200, linewidths=3, label='Centroids')
plt.title('Customer Segments (PCA View)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.legend()

# Plot 2: Feature space visualization
plt.subplot(1, 2, 2)
plt.scatter(df['Annual_Spending'], df['Avg_Order_Value'], c=cluster_labels, cmap='viridis', alpha=0.6)
plt.title('Customer Segments (Annual Spending vs Avg Order Value)')
plt.xlabel('Annual Spending ($)')
plt.ylabel('Average Order Value ($)')

plt.tight_layout()
plt.show()

## Customer Segment Profiles


In [ ]:
#| fig-cap: "Detailed analysis of customer segments"

# Analyze each cluster
cluster_summary = df.groupby('Cluster').agg({
    'Annual_Spending': ['mean', 'std'],
    'Avg_Order_Value': ['mean', 'std'],
    'Total_Orders': ['mean', 'std'],
    'Customer_ID': 'count'
}).round(2)

cluster_summary.columns = ['Avg_Annual_Spending', 'Std_Annual_Spending', 
                          'Avg_Order_Value', 'Std_Order_Value',
                          'Avg_Total_Orders', 'Std_Total_Orders', 'Count']

print("Customer Segment Profiles:")
print("=" * 60)
for cluster in sorted(df['Cluster'].unique()):
    cluster_data = df[df['Cluster'] == cluster]
    print(f"\nSegment {cluster} ({len(cluster_data)} customers):")
    print(f"  • Average Annual Spending: ${cluster_data['Annual_Spending'].mean():.0f}")
    print(f"  • Average Order Value: ${cluster_data['Avg_Order_Value'].mean():.0f}")
    print(f"  • Average Total Orders: {cluster_data['Total_Orders'].mean():.1f}")


| Segment | Percentage | Annual Spending | Average Order Value | Total Orders |
|---------|------------|----------------|---------------------|--------------|
| High-value | 20% | $120 \pm 20$ | $15 \pm 3$ | $450 \pm 80$ |
| Moderate | 40% | $60 \pm 15$ | $8 \pm 2$ | $250 \pm 50$ |
| Bargain | 25% | $25 \pm 8$ | $12 \pm 4$ | $150 \pm 30$ |
| Occasional | 15% | $15 \pm 5$ | $3 \pm 1$ | $300 \pm 60$ |


# Example 2: Plant Classification Based on Morphology

## Plant Species Classification with Iris Data

Let's explore another real-world application using the famous **Iris dataset** from scikit-learn.

::: {.incremental}
* The Iris dataset contains measurements of 150 iris flowers from 3 species
* Each flower has 4 features: sepal length, sepal width, petal length, and petal width
* This is a classic example where we know the true species but want to see if k-means can discover these natural groupings
:::

## Loading and Exploring the Iris Dataset

In [ ]:
#| fig-cap: "Loading and exploring the Iris dataset"

from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print("Iris Dataset Information:")
print("=" * 40)
print(f"Number of samples: {X.shape[0]}")
print(f"Number of features: {X.shape[1]}")
print(f"Feature names: {feature_names}")
print(f"Target names: {target_names}")
print(f"Number of species: {len(target_names)}")

# Display first few samples
print(f"\nFirst 5 samples:")
print(f"{'Sepal Length':<12} {'Sepal Width':<11} {'Petal Length':<12} {'Petal Width':<11} {'Species':<8}")
print("-" * 60)
for i in range(5):
    print(f"{X[i, 0]:<12.1f} {X[i, 1]:<11.1f} {X[i, 2]:<12.1f} {X[i, 3]:<11.1f} {target_names[y[i]]:<8}")

## Visualizing Iris Data in 2D

Let's compare features in pairs. Do you see clusters?

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 6))
axes = axes.ravel()

# Plot all pairwise combinations of features
feature_pairs = [(0, 1), (0, 2), (0, 3), (2, 3)]
titles = ['Sepal Length vs Width', 'Sepal Length vs Petal Length', 
          'Sepal Length vs Petal Width', 'Petal Length vs Width']

for i, (feat1, feat2) in enumerate(feature_pairs):
    axes[i].scatter(X[:, feat1], X[:, feat2], 
                   alpha=0.7, s=50)
    
    axes[i].set_xlabel(feature_names[feat1])
    axes[i].set_ylabel(feature_names[feat2])
    axes[i].set_title(titles[i])
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Visualizing Iris Data in 2D

Feature pairs with ground truth labels.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 6))
axes = axes.ravel()

# Color mapping for species
colors = ['red', 'green', 'blue']

# Plot all pairwise combinations of features
feature_pairs = [(0, 1), (0, 2), (0, 3), (2, 3)]
titles = ['Sepal Length vs Width', 'Sepal Length vs Petal Length', 
          'Sepal Length vs Petal Width', 'Petal Length vs Width']

for i, (feat1, feat2) in enumerate(feature_pairs):
    for species in range(3):
        mask = y == species
        axes[i].scatter(X[mask, feat1], X[mask, feat2], 
                       c=colors[species], label=target_names[species], 
                       alpha=0.7, s=50)
    
    axes[i].set_xlabel(feature_names[feat1])
    axes[i].set_ylabel(feature_names[feat2])
    axes[i].set_title(titles[i])
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

> Question: What observations can you make?


## Try different cluster numbers

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# Calculate WCSS for different numbers of clusters
wcss = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, wcss, 'bo-', linewidth=2, markersize=8)
plt.title('Elbow Method for Iris Dataset')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## K-means Clustering on Iris Data

Results for $k=3$ clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, confusion_matrix
import seaborn as sns

# Perform k-means clustering with k=3 (we know there are 3 species)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X)

# Create confusion matrix
cm = confusion_matrix(y, cluster_labels)

# Convert to DataFrame for better visualization
cm_df = pd.DataFrame(cm, 
                    index=target_names, 
                    columns=[f'Cluster {i}' for i in range(3)])

print(f"\nConfusion Matrix:")
print("Rows: True species, Columns: Predicted clusters")
print(cm_df)

# Visualize clustering results in 2D
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: True species
scatter1 = axes[0].scatter(X[:, 2], X[:, 3], c=y, cmap='viridis', alpha=0.7, s=50)
axes[0].set_xlabel('Petal Length')
axes[0].set_ylabel('Petal Width')
axes[0].set_title('True Species')
axes[0].grid(True, alpha=0.3)

# Add species labels
for i, species in enumerate(target_names):
    mask = y == i
    centroid_x = X[mask, 2].mean()
    centroid_y = X[mask, 3].mean()
    axes[0].annotate(species, (centroid_x, centroid_y), 
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=10, fontweight='bold')

# Plot 2: K-means clusters
scatter2 = axes[1].scatter(X[:, 2], X[:, 3], c=cluster_labels, cmap='viridis', alpha=0.7, s=50)
axes[1].scatter(kmeans.cluster_centers_[:, 2], kmeans.cluster_centers_[:, 3], 
               c='red', marker='x', s=200, linewidths=3, label='Centroids')
axes[1].set_xlabel('Petal Length')
axes[1].set_ylabel('Petal Width')
axes[1].set_title(f'K-means Clusters')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Analysis

In [ ]:
#| fig-cap: "Detailed analysis of k-means clustering performance"

# Analyze cluster assignments
print("Cluster Analysis:")
print("=" * 50)

for cluster in range(3):
    mask = cluster_labels == cluster
    cluster_data = X[mask]
    true_species = y[mask]
    
    # Find the most common species in this cluster
    species_counts = np.bincount(true_species)
    dominant_species = np.argmax(species_counts)
    purity = species_counts[dominant_species] / len(true_species)
    
    print(f"\nCluster {cluster}:")
    print(f"  • Size: {len(cluster_data)} samples")
    print(f"  • Dominant species: {target_names[dominant_species]} ({purity:.1%})")
    print(f"  • Average petal length: {cluster_data[:, 2].mean():.1f} cm")
    print(f"  • Average petal width: {cluster_data[:, 3].mean():.1f} cm")
    print(f"  • Average sepal length: {cluster_data[:, 0].mean():.1f} cm")
    print(f"  • Average sepal width: {cluster_data[:, 1].mean():.1f} cm")

## Key Insights:

* K-means successfully identified the three iris species
* Setosa is perfectly separated from the other species
* Versicolor and Virginica show some overlap, which is realistic
* Petal measurements are more discriminative than sepal measurements
* Overall clustering quality pretty good!


## Why K-means Works Well on Iris Data

* **Well-separated clusters**: The three iris species have distinct characteristics
* **Spherical cluster shapes**: Each species forms roughly spherical groups in feature space
* **Appropriate feature scaling**: All measurements are in the same units (centimeters)
* **A Priori knowledge**: We know the true number of species (cluster number)

# Example 3: k-means image compression

## k-means image compression

Here is a simple example of how $k$-means can be used to reduce
color space and compress data.

Consider the following image.  

* Each color in the image is represented by an integer.  
* Typically we might use 24 bits for each integer (8 bits for R, G, and B).
    
![](figs/L6-annie19980405.jpg){width=350 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Now find $k=16$ clusters of pixels in three dimensional $(R, G, B)$ space
and replace each pixel by its cluster center.

Because there are 16 centroids, we can represent by a 4-bit mapping 
for a compression ratio of $24/4=6\times$.
    
![](figs/L6-annie_016.png){width=500 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Here we cluster into 8 groups (3 bits) for a compression ratio $24/3=8\times$.
    
![](figs/L6-annie_008.png){width=500 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Here we cluster into 4 groups (2 bits) for a compression ratio around $24/2=12\times$.
    
![](figs/L6-annie_004.png){width=500 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Finally, we use 1 bit (2 color groups) for a compression ratio of $24\times$.
    
![](figs/L6-annie_002.png){width=500 fig-align="center"}



## Recap and Next

Today we covered:

* $k$-means clustering
* strengths and weaknesses
* Importance of initialization and cluster number
* Feature scaling
* Example applications:
  * Customer segmentation
  * Plant species classification with Iris data
  * Image color compression

Coming up next, we'll look at some practical aspects of applying $k$-means.